In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict

In [2]:
curr_dir = !pwd
filename = "ALL-biolipsummary-mindist.txt"
shilpa_binding_scores = pd.read_csv(curr_dir[0]+"/"+filename, sep='\t')

In [3]:
#Filter according to score
score_filtered = shilpa_binding_scores[shilpa_binding_scores["WeightedScore"] > 0.1]
#Filter according to number of instances
score_instance_filtered = score_filtered[score_filtered["NumInstances"] >= 10]

In [4]:
%%time
#Filter according to ligands
ligand_filter_idx = []
#Compose a list of Trues for the indices of "real ligands"
for i in range(score_instance_filtered.shape[0]):
    ligand = score_instance_filtered.iloc[i]["LigandType"]
    try:
        if ((ligand.find("PEPTIDE_") >= 0)  or (ligand.find("ION_") >= 0) or (ligand.find("DNABASE_") >= 0) or (ligand.find("RNABASE_") >= 0) or (ligand.find("METABOLITE_") >= 0)):
            ligand_filter_idx.append(True)
        else:
            ligand_filter_idx.append(False)
    except:
        ligand_filter_idx.append(False)


score_instance_ligand_filtered = score_instance_filtered[ligand_filter_idx]

CPU times: user 24.9 s, sys: 264 ms, total: 25.1 s
Wall time: 24.8 s


In [13]:
%%time
domains_ids = (score_instance_ligand_filtered["#HmmID"].unique()).tolist()
domains_binding_dict = defaultdict(dict)
for domain_id in domains_ids:
    domain_name = domain_id[domain_id.find("_")+1:]
    curr_binding_info = score_instance_ligand_filtered[score_instance_ligand_filtered["#HmmID"] == domain_id]
    domains_binding_dict[domain_name]["scores"] = []
    domains_binding_dict[domain_name]["states"] = []
    for index, row in curr_binding_info.iterrows():
        state = row["MatchState"]
        score = row["WeightedScore"]
        domains_binding_dict[domain_name]["scores"].append(score)
        domains_binding_dict[domain_name]["states"].append(state) 

CPU times: user 5.78 s, sys: 8 ms, total: 5.79 s
Wall time: 5.78 s


In [15]:
with open(curr_dir[0]+"/domains_binding_dict.pik", 'wb') as handle:
    pickle.dump(domains_binding_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
positions = []
for x in domains_binding_dict["zf-C2H2"]:
    positions.append(x["state"])

In [10]:
set(positions)

{3, 6, 12, 14, 15, 17, 18, 19, 23}